# ILD CNN prediction part
## this is run on the full patient data
    the model is rebuilt
    the weights are loaded from ILD_CNN_model_weights

In [12]:
import argparse
import numpy as np
import sklearn.metrics as metrics
import cPickle as pickle
import json
from keras.models import model_from_json
import sys
import cv2

from keras.models import Sequential
from keras.utils import np_utils 
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D,AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU

In [13]:
import ild_helpers as H
import cnn_model4 as CNN4

In [14]:
args         = H.parse_args()                          
train_params = {
     'do' : float(args.do) if args.do else 0.5,        
     'a'  : float(args.a) if args.a else 0.3,          # Conv Layers LeakyReLU alpha param [if alpha set to 0 LeakyReLU is equivalent with ReLU]
     'k'  : int(args.k) if args.k else 4,              # Feature maps k multiplier
     's'  : float(args.s) if args.s else 1,            # Input Image rescale factor
     'pf' : float(args.pf) if args.pf else 1,          # Percentage of the pooling layer: [0,1]
     'pt' : args.pt if args.pt else 'Avg',             # Pooling type: Avg, Max
     'fp' : args.fp if args.fp else 'proportional',    # Feature maps policy: proportional, static
     'cl' : int(args.cl) if args.cl else 5,            # Number of Convolutional Layers
     'opt': args.opt if args.opt else 'Adam',          # Optimizer: SGD, Adagrad, Adam
     'obj': args.obj if args.obj else 'ce',            # Minimization Objective: mse, ce
     'patience' : args.pat if args.pat else 5,         # Patience parameter for early stoping
     'tolerance': args.tol if args.tol else 1.005,     # Tolerance parameter for early stoping [default: 1.005, checks if > 0.5%]
     'res_alias': args.csv if args.csv else 'res'      # csv results filename alias
}

In [15]:
model = H.load_model()

In [16]:
model.compile(optimizer='Adam', loss=CNN4.get_Obj(train_params['obj']))

In [23]:
X_predict = pickle.load( open( "X_predict_107.pkl", "rb" ) )

In [24]:
# adding a singleton dimension and rescale to [0,1]
X_predict = np.asarray(np.expand_dims(X_predict,1))/float(255)

In [25]:
# predict and check classification and probabilities are the same

classes = model.predict_classes(X_predict, batch_size=10)
proba = model.predict_proba(X_predict, batch_size=10)

463/463 [==============================] - 6s     


In [26]:
classes

array([1, 4, 4, 4, 1, 1, 4, 4, 1, 4, 4, 1, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 5, 1, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4,
       2, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 5, 4,
       5, 5, 4, 4, 4, 4, 5, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 2, 4, 4, 5, 5,
       5, 5, 4, 4, 2, 5, 5, 4, 1, 5, 5, 5, 5, 3, 4, 4, 5, 5, 2, 4, 4, 5, 5,
       5, 5, 5, 5, 4, 2, 5, 5, 5, 4, 5, 5, 4, 5, 5, 5, 3, 5, 1, 3, 4, 4, 5,
       5, 2, 4, 4, 2, 5, 1, 5, 5, 5, 5, 5, 4, 5, 2, 5, 2, 5, 4, 5, 1, 1, 4,
       5, 1, 4, 3, 4, 3, 4, 5, 5, 1, 5, 4, 4, 4, 1, 5, 5, 5, 5, 5, 5, 5, 4,
       5, 5, 5, 5, 5, 1, 5, 5, 1, 5, 1, 3, 4, 3, 4, 1, 5, 5, 4, 2, 2, 1, 5,
       4, 5, 5, 5, 2, 5, 5, 1, 1, 5, 2, 5, 5, 5, 5, 1, 5, 2, 4, 4, 4, 4, 4,
       1, 5, 5, 4, 4, 5, 5, 2, 5, 1, 1, 5, 2, 5, 5, 5, 5, 2, 2, 5, 5, 5, 5,
       5, 5, 3, 3, 4, 1, 4, 5, 4, 1, 5, 4, 4, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 1, 1, 5, 5, 5, 2, 5, 5, 5, 4, 1, 1, 3, 1, 1, 1, 1, 1, 5,
       4, 2,

In [27]:
proba

array([[  4.06456664e-08,   4.83859867e-01,   5.56275547e-02,
          2.54847855e-05,   4.60416943e-01,   7.00841993e-05],
       [  1.35587434e-05,   5.47280833e-02,   3.77012253e-01,
          2.64885928e-03,   5.38528144e-01,   2.70691346e-02],
       [  2.40960111e-13,   3.33996053e-04,   5.43991227e-05,
          2.43857739e-05,   9.99586999e-01,   2.31058920e-07],
       ..., 
       [  7.29719352e-04,   7.52775490e-01,   1.44615486e-01,
          2.17827343e-04,   4.29083407e-03,   9.73706394e-02],
       [  2.04139762e-03,   9.61278796e-01,   1.44251226e-03,
          1.04132923e-05,   9.62287595e-05,   3.51306498e-02],
       [  1.55441856e-04,   9.32932794e-01,   1.26936575e-02,
          6.18283157e-05,   4.95925965e-03,   4.91970181e-02]])

In [28]:
pickle.dump(classes, open( "predicted_classes_107.pkl", "wb" ))
pickle.dump(proba, open( "predicted_probabilities_107.pkl", "wb" ))